#### Workflow:
Each 'hash_patient_id' is processed separately to make the process efficient.

1. Create observations with 'create_data_points':
 - an observation is defined as a data point such that all signals 'peep', 'fio2', 'po2'
 are measured within one hour. The mean of all measurements within the hour is taken
 and the hour is rounded up to the nearest hour.
 - As an example if for a single 'hash_patient_id' we have three measurements:
 'peep' measured at 12:50, 'fio2' measured at 12:10 and po2 measured at 12:20, 12:40, 13:00
 then these measurements will create one data point: 13:00.
 - As a second example if fio2 would not have been measured between 12:01 and 13:00. Then
 we would discard all observations that happened between 12:01 and 13:00 for all signals.
 - Notebook works with a single patient and hence '_create_data_points_batch' to make
testing faster. Consider changing the function name to 'create_observations'.

2. (to be implemented) Function 'select_cohort' selects the cohort by checking the
inclusion and exclusion criteria. (First to do is to plot all values as a histogram,
possibly without discarding the non measured ones.)

3. Create a table with the treatment. Function 'get_proning' creates a table with
proning sessions. This table will be used to split the observations into control, treated,
outcome control, outcome treated.

load blood gas points (create_data_points)
 -> drop ids that are not eligible (check_inclusion(drop = True))
 -> split points into (control - not proned and not proned after
                       measurement_control - not proned and wasn't proned in the past, may be proned in the future
                       trated - not proned will be proned after
                       measurement_treated - proned at the moment)


In [ ]:
%reset

In [ ]:
import os, sys, random

import pandas as pd
import numpy as np
import swifter

import pacmagic
import pacmagic_icu

from importlib import reload
from data_warehouse_utils.dataloader import DataLoader

os.chdir('/home/adam/files/causal_inference')
os.getcwd()

from causal_inference.experiment_generator.create_observations import create_data_points
from causal_inference.experiment_generator.create_treatment import get_proning_table
from causal_inference.experiment_generator.create_treatment import add_treatment
from causal_inference.experiment_generator.create_inclusion_criteria import get_inclusion_data

In [ ]:
# Reloads packages
os.chdir('/home/adam/files/causal_inference')
os.getcwd()

reload(sys.modules['causal_inference.experiment_generator.create_observations'])
reload(sys.modules['causal_inference.experiment_generator.create_treatment'])
reload(sys.modules['causal_inference.experiment_generator.create_inclusion_criteria'])


from causal_inference.experiment_generator.create_observations import create_data_points
from causal_inference.experiment_generator.create_treatment import get_proning_table
from causal_inference.experiment_generator.create_treatment import add_treatment
from causal_inference.experiment_generator.create_inclusion_criteria import get_inclusion_data

In [ ]:
dl = DataLoader()

In [ ]:
df_measurements = create_data_points(dl)

In [ ]:
df_measurements.head()

In [ ]:
df_measurements.info()

In [ ]:
os.chdir('/home/adam/files/data')
os.getcwd()

In [ ]:
df_measurements.to_csv('blood_gas_measurements.csv', index=False)

In [ ]:
df_treatment = get_proning_table(dl)

In [ ]:
df_treatment.info()

In [ ]:
df_treatment.head()

In [ ]:
os.chdir('/home/adam/files/data')
os.getcwd()

In [ ]:
df_treatment.to_csv('prone_sessions.csv', index=False)

In [ ]:
df_observations = add_treatment(df_treatment)

In [ ]:
os.chdir('/home/adam/files/data')
df_measurements = pd.read_csv('blood_gas_measurements.csv')
df_measurements.head()

In [ ]:
df_observations = get_inclusion_data(df_observations, df_measurements)

In [ ]:
df_observations.head()

In [ ]:
df_treatment.describe()

In [ ]:
df_treatment[df_treatment.effective_value == 'prone'].describe()

In [ ]:
df_treatment[df_treatment.effective_value == 'prone'][df_treatment.duration_hours <= 96].describe()

In [ ]:

df_treatment.dtypes